<a href="https://colab.research.google.com/github/Ali-MH-Mansour/Bag-of-concepts/blob/main/Code/Bag_of_concepts_Vdhya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Connect to the Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Functions for the data set preprocessing

In [3]:
import re
import string
import pandas as pd
import numpy as np
import math
from numpy import array 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from sklearn.cluster import KMeans
from collections import Counter 
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
#-------------------------------------------------------------------------------
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
#-------------------------------------------------------------------------------
def preprocess_text(text):
    text = re.sub(r'\d+' , '', text)
    text = remove_punctuation(text)
    text = text.lower()
    text = text.strip()
    text = re.sub(r'\b\w{1,2}\b', '', text)
    tokens = toknizing(text)
    pos_map = {'J': 'a', 'N': 'n', 'R': 'r', 'V': 'v'}
    pos_tags = pos_tag(tokens)
    lemmatiser = WordNetLemmatizer()
    tokens = [lemmatiser.lemmatize(t.lower(), pos=pos_map.get(p[0], 'v')) for t, p in pos_tags]
    return tokens

#--------------------------------------------------------------------------------------------
def remove_punctuation(text):
     punctuations = '''!()-[]{};:'"\,<>./?@#$%^+&*_~'''
     no_punct = ""
     for char in text:
        if char not in punctuations:
            no_punct = no_punct + char
     return no_punct

#--------------------------------------------------------------------------------------------
def toknizing(text):
  stop_words = set(stopwords.words('english'))
  tokens = word_tokenize(text)
  result = [i for i in tokens if not i in stop_words]
  return result
#-------------------------------------------------------------------------------
def cosine_distance(u, v):
  return np.dot(u, v) / (math.sqrt(np.dot(u, u)) * math.sqrt(np.dot(v, v)))
#-------------------------------------------------------------------------------
!pip install soyclustering
from soyclustering import SphericalKMeans
from scipy.sparse import csr_matrix
#-------------------------------------------------------------------------------
def spherical_kmeans(n_clusters_, list_of_words_vectors,sim_digree):
  embeddings_matrix_csr = csr_matrix(list_of_words_vectors)
  spherical_kmeans = SphericalKMeans( max_similar=sim_digree, init='similar_cut', 
      n_clusters = n_clusters_)
  labels = spherical_kmeans.fit_predict(embeddings_matrix_csr)
  centers = spherical_kmeans.cluster_centers_
  return labels , centers

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


##Data sets Downolad

In [4]:
BBC_file_path="/content/drive/........./Data/bbc-text.csv"
data_DF = pd.read_csv(BBC_file_path)

# rename columns
data_DF.columns.values[0] = "class"
data_DF.columns.values[1] = "document"

data_DF.head()

,class,document
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [5]:
# Factorize class names to numbers
data_DF['class'] = pd.factorize(data_DF['class'])[0]
data_DF['document'] = data_DF.document.apply(lambda x: ' '.join(preprocess_text(x)))
data_DF.head()

,class,document
0,0,future hand viewer home theatre system plasma ...
1,1,worldcom bos leave book alone former worldcom ...
2,2,tiger wary farrell gamble leicester say rush m...
3,2,yeading face newcastle cup premiership side ne...
4,3,ocean twelve raid box office ocean twelve crim...


## Train Fasttext model on BBC data set

In [6]:
# Train fast text model on your dataset
import gensim
from gensim.models import FastText
documents_as_list = data_DF['document'].tolist()
doc_as_tokens =[]
for doc in documents_as_list:
  doc_as_tokens.append(doc.split())

In [7]:
model = FastText(doc_as_tokens, size=300, window=15, min_count=1, workers=4,sg=1)

## Create Dictionary of words 

In [9]:
dic_w_v = {}
all_doc_emb = []
for doc in doc_as_tokens:
  doc_emb = []
  for w in doc:
    try :
      emb = model.wv[w]
      doc_emb.append(emb)
      dic_w_v.update({w : emb})
    except: 
      continue;
  all_doc_emb.append(doc_emb)

## Clustering the words (concepts extraction)

In [10]:
list_of_words_vectors = list(dic_w_v.values())

In [11]:
# number of clusters
K_clusters= 300
sim_digree=0.4
######### Clustering
labels , centers = spherical_kmeans(K_clusters , list_of_words_vectors, sim_digree)
print("The clusters details", Counter(labels), "\n")

The clusters details Counter({54: 274, 24: 255, 206: 245, 74: 219, 58: 219, 69: 212, 145: 207, 113: 203, 97: 192, 276: 191, 148: 191, 71: 191, 195: 191, 2: 185, 233: 185, 280: 183, 179: 176, 189: 172, 25: 172, 141: 169, 264: 168, 102: 168, 297: 167, 57: 165, 35: 163, 17: 161, 135: 159, 39: 156, 60: 155, 126: 154, 48: 154, 222: 152, 36: 151, 117: 148, 244: 147, 118: 146, 254: 144, 105: 143, 63: 141, 227: 141, 187: 141, 286: 140, 89: 140, 75: 139, 109: 139, 270: 138, 267: 134, 284: 134, 14: 133, 4: 133, 295: 132, 186: 132, 178: 131, 192: 131, 98: 127, 77: 127, 131: 126, 293: 126, 46: 125, 99: 124, 240: 123, 27: 123, 299: 123, 292: 122, 261: 122, 164: 121, 210: 121, 31: 120, 66: 120, 139: 120, 49: 118, 161: 117, 175: 117, 155: 117, 230: 116, 91: 116, 262: 116, 165: 115, 176: 115, 294: 115, 268: 112, 138: 111, 211: 111, 228: 111, 253: 110, 237: 109, 90: 108, 81: 107, 44: 107, 88: 106, 266: 105, 10: 104, 160: 104, 41: 104, 289: 104, 291: 102, 120: 101, 128: 100, 80: 100, 144: 100, 217: 100,

# Bag of weighted Concepts

### Documents vectorization

In [12]:
#خاص بحقيبة الكلمات الاصلية
def BOC(embeddeings_document,alpha,centers):
  concept_occurence = 0
  ## intilize doc array
  CF_vector = np.zeros(len(centers)).tolist()
  for cnt in range(len(centers)):
    for emb_word in embeddeings_document:
      Wsim= cosine_distance(emb_word , centers[cnt])
      if Wsim  > alpha:
        concept_occurence = concept_occurence + 1 #for tf
        CF_vector[cnt] = CF_vector[cnt] + 1;
      else:
        continue;    
  if concept_occurence !=0:
    for i in CF_vector:
      i = i / concept_occurence  #count/
  return CF_vector
#-------------------------------------------------------------
def CF_IDF_BOC(documents_embeddings , alpha , centers):
  CFs = []
  counter = 0
  Df = np.zeros(len(centers)).tolist() # بطول العناقيد
  for doc in documents_embeddings: 
    CF = BOC(doc , alpha , centers )
    for i in CF: 
      if i > 0:
        Df[counter] = Df[counter] + 1 #  i[M-1]
      counter = counter+1
    counter = 0
    CFs.append(CF)
  #########  calculate IDF
  IDF=[]
  for df in Df:
    if df!=0:
      N=len(documents_embeddings)
      IDF.append( math.log10( (N+1) /df ) )
    else:
      IDF.append(0)  
  #Calculate Cf-IDF
  for i in range(N):
    l=len(CFs[i])
    for j in range(l):
       CFs[i][j] = CFs[i][j] * IDF[j]  #cf*idf
  return CFs

In [ ]:
# Thershold similarity values
alpha = 0.4

final_embeddings  = CF_IDF_BOC(all_doc_emb , alpha , centers)

##Train SVM for dataset calssification

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import f1_score
#-------------------------------------------------------------------------------
classes = data_DF['class'].tolist()
x_train , x_test , y_train , y_test = train_test_split(final_embeddings , classes , test_size = 0.30 , random_state = 0)
svm =  SVC(kernel='rbf', random_state=44).fit(x_train , y_train) 

##**Evaluation**

In [ ]:
y_pred=svm.predict(x_test)
F1 = f1_score(y_test,y_pred, average="micro")

print("The micro average-weighted f1 score = ", F1)

The micro average-weighted f1 score =  0.8667664670658682


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 89,  13,   0,   5,  10],
       [  1, 139,   2,   1,   9],
       [  0,   0, 174,   0,   3],
       [  1,   6,  16,  83,   3],
       [  2,  25,  11,   2,  73]])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.76      0.85       117
           1       0.76      0.91      0.83       152
           2       0.86      0.98      0.92       177
           3       0.91      0.76      0.83       109
           4       0.74      0.65      0.69       113

    accuracy                           0.84       668
   macro avg       0.85      0.81      0.82       668
weighted avg       0.84      0.84      0.83       668

